# Ámbito 2: Calidad de la Argumentación
## Dimensión: Tesis

En este notebook se probarán Prompts de GPT para la corrección de la dimensión Tesis correspondiendiente al Ámbito 2 de la Rúbrica del Proyecto de Escritura. También se agregarán las demás dimensiones del Ámbito 2.

In [ ]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

textos = textos  # Para pruebas, limitar a 2 textos

prompt_tesis_template = prompt_tesis_template = """Analiza el texto argumentativo que te entregaré y determina EXCLUSIVAMENTE los elementos necesarios para evaluar según los siguientes criterios y definiciones:

### Definiciones:
- **Tesis:** Afirmación central y explícita que el autor defiende sobre el tema. Es un enunciado completo, subrayable, con verbo principal conjugado y que expresa una posición defendible (puede incluir explícita o implícitamente un juicio de valor, pero NO es obligatorio que lo haga mediante un adjetivo). Representa la idea central que guía la argumentación.
- **Postura explícita:** Implica una declaración de acuerdo o desacuerdo personal EXPRESA. Ejemplos: “Yo no estoy de acuerdo con…”, “A mí no me parece…”, “Yo no creo…”. En este caso la postura es aislable y subrayable en el texto.
- **Postura implícita:** Implica una declaración de acuerdo o desacuerdo personal IMPLÍCITA, la cual solo puede deducirse a partir de la lectura global del escrito. Una postura implícita no es aislable ni subrayable de forma directa y no deben poder detectarse dos posturas contrarias igualmente desarrolladas dentro de la lectura global.
- **Postura:** Término general que incluye tanto postura explícita como implícita. Se trata de una declaración inicial o básica de posición frente al tema, sin argumentación desarrollada; es como una “tesis embrionaria”: muestra hacia dónde va el autor pero sin formularse como proposición argumentativa completa.

### Criterios de evaluación TESIS:
- **3 puntos (Tesis explícita):** Se presenta una tesis EXPLÍCITA, clara, con verbo principal conjugado y posible de subrayar.
- **2 puntos (Postura explícita):** Se presenta una postura EXPLÍCITA, aislable/subrayable.
- **1 punto (Postura implícita):** Se presenta una postura IMPLÍCITA, deducible de la lectura global.
- **0 puntos (Sin tesis ni postura):** Ninguno de los casos anteriores. No es posible identificar una tesis ni una postura en el texto.

### Tu respuesta debe ser un JSON válido con la siguiente estructura EXACTA:
{{
  "tesis_o_postura": "Texto de la tesis o postura identificada (o 'No identificada' si no hay)",
  "clasificación": "tesis explícita / postura explícita / postura implícita / sin tesis",
  "nivel_logro": 3 | 2 | 1 | 0,
  "justificación": "Explica brevemente por qué asignaste esta clasificación y nivel"
}}

### Texto a evaluar:
{essay}

### INSTRUCCIONES CRÍTICAS:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
- Identifica la tesis o postura en la forma más breve y precisa posible.
- Si no hay tesis ni postura, indica claramente "No identificada".
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_tesis = prompt_tesis_template.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_tesis}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "tesis.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\tesis.json


## Dimensión: Calidad de la argumentación

In [33]:
# Directorio del proyecto (un nivel arriba de notebooks)
proyecto_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ruta absoluta al archivo JSON
ruta_json = os.path.join(proyecto_dir, "resultados", "02", "tesis.json")

with open(ruta_json, "r", encoding="utf-8") as f:
    tesis_json = json.load(f)

# 2️⃣ Crear la lista de tesis detectadas
tesis_list = [item["resultado"]["tesis_o_postura"] for item in tesis_json]

In [37]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_argumentos_template = """Analiza el texto argumentativo que te entregaré y determina EXCLUSIVAMENTE los elementos necesarios según los siguientes criterios y definiciones. Te entregaré la tesis detectada previamente, para que relaciones los elementos con ella:

### Definiciones de ARGUMENTOS:
- **Argumentos sólidos:** Datos, hechos, cifras, argumentos causales, deductivos, inductivos y opiniones fundadas que se relacionan lógicamente con la tesis.
- **Argumentos débiles:** Generalizaciones, argumentos de autoridad, falacias y otros recursos poco fundamentados.
- **Argumentos afectivo-persuasivos:** Sentimientos, sensaciones, opiniones infundadas, apelación a la emotividad del lector (temores, deseos inconscientes, supersticiones, falsas creencias, etc.).

### Niveles de logro por ARGUMENTOS:
- **3 puntos:** Se presentan al menos tres argumentos sólidos relacionados con la tesis planteada, sin incluir argumentos afectivo-persuasivos.
- **2 puntos:** Se presentan dos argumentos sólidos y al menos un argumento débil que se relacionan con la tesis planteada, sin incluir argumentos afectivo-persuasivos.
- **1 punto:** 
  - Se presentan solo dos argumentos sólidos relacionados con la tesis planteada, sin incluir argumentos débiles ni afectivo-persuasivos,  
  O  
  - Se presenta un argumento sólido y al menos dos argumentos débiles relacionados con la tesis, sin incluir afectivo-persuasivos,  
  O  
  - Se presentan al menos tres argumentos débiles relacionados con la tesis, sin incluir afectivo-persuasivos.
- **0 puntos:** Se presenta al menos un argumento afectivo-persuasivo que se relaciona con la tesis,  
O  
No se presentan argumentos (el texto es expositivo, un punteo de oraciones, formulación de preguntas, etc.).

### Tu respuesta debe ser un JSON válido con la siguiente estructura EXACTA:
{{
  "argumentos_solidos": ["Argumento sólido 1", "Argumento sólido 2", ...],
  "argumentos_debiles": ["Argumento débil 1", "Argumento débil 2", ...],
  "argumentos_afectivo_persuasivos": ["Argumento afectivo 1", "Argumento afectivo 2", ...],
  "nivel_logro_argumentacion": 3 | 2 | 1 | 0,
  "justificación": "Explica brevemente por qué asignaste este nivel"
}}

### Tesis detectada previamente:
{detected_thesis}

### Texto a evaluar:
{essay}

### INSTRUCCIONES CRÍTICAS:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
- Si no hay argumentos de algún tipo, devuelve un array vacío [] en ese campo.
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_argumentos = prompt_argumentos_template.format(essay=texto, detected_thesis=tesis_list[i-1])

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_argumentos}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "argumentacion.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\argumentacion.json


## Dimensión: Contraargumentación

In [ ]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_contraargumentos_template = """Analiza el texto argumentativo que te entregaré y determina EXCLUSIVAMENTE los elementos necesarios según los siguientes criterios y definiciones. Te entregaré la tesis detectada previamente, para que relaciones los elementos con ella:

### Definiciones de CONTRAARGUMENTACIÓN:
- **Contraargumentación:** Se presenta al menos un contrargumento que contiene una objeción a la tesis planteada, se analiza detalladamente (se presenta el contrargumento y se explica su sentido) y luego se refuta.
- **Mencionar un contrargumento:** Hacer referencia a algún argumento de la postura opuesta.
- **Desarrollar un contrargumento:** Explicarlo o fundamentarlo desde la postura opuesta.
- **Rebatir o refutar:** Plantear argumentos para oponerse al contrargumento planteado.

### Niveles de logro por CONTRAARGUMENTACIÓN:
- **3 puntos:** Se menciona, desarrolla y rebate al menos un contrargumento.
- **2 puntos:** Se menciona y rebate al menos un contrargumento.
- **1 punto:** Se desarrolla o menciona al menos un contrargumento, pero no se rebate.
- **0 puntos:** NO se mencionan contrargumentos.

### Tu respuesta debe ser un JSON válido con la siguiente estructura EXACTA:
{{
  "contrargumentos": [
    {{
      "texto": "Texto completo del contrargumento",
      "mencionado": True | False,
      "desarrollado": True | False,
      "rebatido": True | False
    }},
    ...
  ],
  "nivel_logro_contraargumentacion": 3 | 2 | 1 | 0,
  "justificación": "Explica brevemente por qué asignaste este nivel"
}}

### Tesis detectada previamente:
{detected_thesis}

### Texto a evaluar:
{essay}

### INSTRUCCIONES CRÍTICAS:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
- Cada contrargumento debe aparecer **solo una vez** en la lista, con sus campos booleanos correspondientes.
- Si no hay contrargumentos, devuelve un array vacío [] en "contrargumentos".
"""

resultados = []

for i, texto in enumerate(textos, start=1):
    prompt_contraargumentos = prompt_contraargumentos_template.format(essay=texto, detected_thesis=tesis_list[i-1])

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_contraargumentos}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "02")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "contraargumentos.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\02\contraargumentos.json
